### Building Question answering system

For  this work , we will build our question answering system , we will leverage the [deepstack framework]() to build the components of our system.

Here are the followign components of our system:
-  The index store 
- The document store 
- The search pipeline with a Retrieval and a Reader model.

We will be leveraging the tutorials provided by deepstack to build our system.

For this work we will leverage [this  tutorial:](https://github.com/deepset-ai/haystack/blob/master/tutorials/Tutorial6_Better_Retrieval_via_DPR.ipynb) 

### Building the dense passage retrieval

To build the dataset for the dense passage retrieval , we will be using the approach suggested by [this tutorial](https://huggingface.co/etalab-ia/dpr-question_encoder-fr_qa-camembert) which use the cambert model .

For each question , we have a single positive context , the paragraph where the answer to the question is located and n hard negavives contexts that are the the top - k canditates taht does not contain the answer, to the question. to retrieve the negative context they will use the bm25 retrieval model.

Before training the retrieval model we will have to build the document store and save the document as units of retrieval to the store.

### Building the dataset

In [2]:
import pandas as pd
import numpy as np

In [3]:
from pathlib import Path
DATA_PATH = Path.cwd().joinpath("data")
assert DATA_PATH.exists(), "the data path does not exist"
TEXT_DATA_FOLDER = DATA_PATH.joinpath("corpus", "drc-news-txt")
assert TEXT_DATA_FOLDER.exists(), "the text data folder does not exist"

In [4]:
data_file_path = DATA_PATH.joinpath("corpus", "raw", 'drc-news-raws.csv')

In [5]:
data = pd.read_csv(data_file_path, names=["content", "posted_at"])

In [5]:
data.shape

(140638, 2)

In [6]:
data.head()

,content,posted_at
0,Les membres de la Commission tarifaire viennen...,2022-09-05 00:00:00
1,Les membres de la Commission tarifaire so...,2022-04-05 00:00:00
2,Vodacom Congo vient de signer un partenariat a...,2022-04-23 00:00:00
3,"Le sélectionneur des Léopards de la RDC, Hectó...",2022-03-05 00:00:00
4,Le protocole d’accord était déjà signé entre l...,2022-11-05 00:00:00


In [7]:
for index, row in enumerate(data.sample(1000).loc[:, ["content"]].values):
   with open(TEXT_DATA_FOLDER.joinpath(f"{index}.txt"), "w") as f:
       f.write(row[0])

In [9]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
from haystack.utils import convert_files_to_docs

In [10]:
all_docs = convert_files_to_docs(dir_path=TEXT_DATA_FOLDER)

INFO - haystack.utils.preprocessing -  Converting /Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/corpus/drc-news-txt/289.txt
INFO - haystack.utils.preprocessing -  Converting /Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/corpus/drc-news-txt/504.txt
INFO - haystack.utils.preprocessing -  Converting /Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/corpus/drc-news-txt/262.txt
INFO - haystack.utils.preprocessing -  Converting /Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/corpus/drc-news-txt/276.txt
INFO - haystack.utils.preprocessing -  Converting /Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/corpus/drc-news-txt/510.txt
INFO - haystack.utils.preprocessing -  Converting /Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/corpus/drc-news-txt/538.txt
INFO - haystack.utils.preprocessing -  Converting /Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/corpus/drc-news-txt/

In [11]:
all_docs[0]

<Document: {'content': 'À l’approche de la désignation des nouveaux animateurs de la Commission Electorale Nationale Indépendante (CENI), les députés nationaux vont enfin démarrer l’examen de la proposition de loi organique qui vise non seulement la modification de la loi portant organisation et fonctionnement de la centrale électorale, mais aussi la réforme du système électoral de la RDC. Cette proposition a été déposée depuis le 8 août 2019 par le Député national, Christophe Lutundula.Cet élu de Katakokombe avait motivé sa démarche par le fait que les trois cycles électoraux précédents ont connu des violences avec mort d’hommes ainsi que d’innombrables contentieux devant les tribunaux. Outre ces violences, Christophe Lutundula avait relevé certains scandales financiers dans lesquels la CENI a été citée et qui suscitent des suspicions quant à la gestion de ses finances.Parmi ses réformes, Lutundula propose un effectif de 12 membres qui doivent composer la Commission électorale nationa

In [12]:
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process(all_docs)

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")

100%|██████████| 1000/1000 [00:00<00:00, 1256.52docs/s]

n_files_input: 1000
n_docs_output: 3828


In [13]:
docs[0]

<Document: {'content': 'À l’approche de la désignation des nouveaux animateurs de la Commission Electorale Nationale Indépendante (CENI), les députés nationaux vont enfin démarrer l’examen de la proposition de loi organique qui vise non seulement la modification de la loi portant organisation et fonctionnement de la centrale électorale, mais aussi la réforme du système électoral de la RDC.', 'content_type': 'text', 'score': None, 'meta': {'name': '289.txt', '_split_id': 0}, 'embedding': None, 'id': '6f746160960651cd7fd47ef8dd6ef6f3'}>

In [15]:
from haystack.document_stores import ElasticsearchDocumentStore

In [176]:
document_store = ElasticsearchDocumentStore(index="drc-news")

In [17]:
document_store.write_documents(docs)

/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/.venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


### Retrieval

With the document store in place , the document store has all the document in it , let build a retriever model that use BM25 to retrieve the document.

In [177]:
from haystack.nodes import BM25Retriever


In [178]:
query= "Qui est le Le gouverneur de la province du Kongo Central"

In [179]:
bm25_retriever = BM25Retriever(document_store=document_store)

In [254]:
bm25_retriever = BM25Retriever(document_store=document_store, all_terms_must_match=False)


In [263]:
custom_query = "Qui es le president de la republique democratique du congo"

In [264]:
bm25_retriever.retrieve(query=custom_query, top_k=10)

this is the retriever query : {'size': '10', 'query': {'bool': {'must': [{'multi_match': {'query': 'Qui es le president de la republique democratique du congo', 'type': 'most_fields', 'fields': ['content'], 'operator': 'OR'}}], 'must_not': [{'wildcard': {'content': {'value': '*félix tshisekedi*'}}}]}}}


[<Document: {'content': 'du Congo)\xa0;-Al Masry (Égypte) vs à USM Alger (Algérie)\xa0;-Rayon Sports (Rwanda) vs Enyimba (Nigéria)\xa0;-AS V.Club (RDC) vs Renaissance Berkane (Maroc)\xa0;Quart de finales Ligue de champions-Mazembe (RDC) vs Primero d’Agosto (Angola)\xa0;-Espérance Tunis (Tunisie) vs ES Sahel (Tunisie),-Al Ahly (Egypyte) vs Horoya,-ES Sétif (Algérie) vs WAC Casablanca (Maroc).EG', 'content_type': 'text', 'score': 0.812827707917524, 'meta': {'_split_id': 5, 'name': '672.txt'}, 'embedding': None, 'id': 'aa25acf40c05c7724e7184dcccf70655'}>,
 <Document: {'content': 'TIBASIMA MBOGEMU ATEENYI, CANDIDAT 2 VICE-PRESIDENT DU SENATHonorable Président du Bureau Provisoire du Senat,Honorables Membres du Bureau Provisoire du Senat,Honorables Sénateurs et Chers collègues,C’est pour nous un honneur de nous présenter et de nous exprimer devant vous ce jour, en vue de solliciter votre confiance, pour que vous daigniez faire de notre modeste personne l’un des membres du Bureau Définitif d

In [ ]:
 "filter": {
                    "regexp":{
                     "content": {
                        "value": ".*~(Ngwabidje).*"
            }
        }}

In [96]:
print('«\xa0«\xa0, fait savoir celui qu’on appelle affectueusement «\xa0«\xa0. Dina BUHAKE')

« « , fait savoir celui qu’on appelle affectueusement « « . Dina BUHAKE
